### Import all required libraries

In [ ]:
import itertools
import os

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.ndimage import gaussian_filter

from mpl_toolkits.mplot3d import Axes3D
import scipy as scipy
from scipy import stats

%matplotlib widget

In [ ]:
x = 0
a = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

c = list(dict.fromkeys(list(map(' '.join, itertools.chain(
    itertools.product(a, a), itertools.product(a, a))))))


df2 = pd.DataFrame(columns=c, index=c)


for col in df2.columns:
    df2[col].values[:] = 0

totals = []

midi = pd.read_csv('data/midi.csv')
for index, row in midi.iterrows():
    try:
        filepath = 'data/pieces/' + row['File Path'] + '.csv'
        df1 = pd.read_csv(filepath, index_col=0)
        totals.append(df1.to_numpy().sum())
        df2 = pd.concat([df1, df2]).groupby(level=0).sum()
    except:
        print(f'Didn\'t work with {filepath}')
df2 = df2.reindex(a)

display("Total Note-Transition Counts For All Pieces")
df2

In [ ]:
print(f'Total Piece Count: {len(os.listdir("data/pieces/"))}')
print('Bigram count ' + '{:,}'.format(df2.to_numpy().sum()))

print("Std = " + str(round(np.std(totals), 2)))
print("Mean = " + str(round(np.mean(totals), 2)))
print("Median = " + str(round(np.median(totals), 2)))

sns.displot(data=totals, color='green', linewidth=0,
            bins=100, kde=False, height=4, aspect=2);

In [ ]:
def plotMatrix(data, annotate=False, smooth=0, color='viridis', save=False, index='0', show=True):
    if annotate == True:
        sns.set(font_scale=1)
    else:
        sns.set(font_scale=1.5)
    
    yticks = data.index
    keptticks = yticks[::int(len(yticks)/10)]
    yticks = ['' for y in yticks]
    yticks[::int(len(yticks)/10)] = keptticks

    xticks = data.columns
    keptticks = xticks[::int(len(xticks)/10)]
    xticks = ['' for y in xticks]
    xticks[::int(len(xticks)/10)] = keptticks

    fig, ax1,  = plt.subplots(figsize=(8, 6), dpi=100)

    data = gaussian_filter(data, sigma=smooth)
    sns.heatmap(data, ax=ax1, linewidth=0, yticklabels=yticks,
                xticklabels=xticks, cmap=color, annot=annotate).set(title=f'n = {index}')

    plt.yticks(rotation=0)
    if save == True:
        plt.savefig(f'data/matrices/matrix{index}.png')
    if show != False:
        plt.show()

In [ ]:
# Converting to a stochastic matrix
df4 = df2
for note in a:
    df4.loc[note] = df2.loc[note] / df2.sum(axis=1)[note]
df4

In [ ]:
# Plotting example matrices
plotMatrix(df4, annotate=True)
plotMatrix(df4)
plotMatrix(df4, smooth=1)